# (5) Functional enrichment of GO terms



---
Michaël Pierrelée, Aix Marseille Univ, CNRS, IBDM, UMR7288, FRANCE - michael.pierrelee@univ-amu.fr

*Apache License 2.0*

---

Perform functional enrichment using GSEAPY for GO terms, a Python implementation of Gene Set Enrichment Analysis (GSEA).

First, transcritps were aggregated for each gene by SuperTranscript. Gene sequences were aligned by BlastN on SwissProt database and GO-annotated on GOA using Blast2GO.

Second, the ranked list was generated. The means of LFC E and S of genes, with FDR a lower than **1e-4**, related to duplicated KOs, were computed before to sum both LFCs in absolute value

Third, a background gene set was built by taking all the genes from Blast2GO with at least one GO term, following Following to Powell (2014) [doi:10.1186/1471-2105-15-146](https://doi.org/10.1186/1471-2105-15-146). These genes were also annotated by the children of these GO terms (e.g. GO terms with broader definitions) using the tree of GO terms generated from the OBO file (downloadable from http://release.geneontology.org/2019-10-07/ontology/go-basic.obo). Reciprocally, each GO term was associated to a list of genes, giving the gene set used for enrichment. When two or more GO terms had an identical gene list, the terms were merged.

Enrichment was performed by GSEAPY with the following parameters: seed at 0, 1000 permutations and minimum and maximum gene set size at 5 and 1000 resp.


## Input

* `data/dea_results/synthesis_deg_results.csv`: synthesis of DEA results with the mean of normalized counts, the LFCs and adjusted p-values of comparisons M vs. F at E and S and the interaction term values (LFC, padj and strength of variation in % and categorized into no significant, low, medium or strong variation).
* `data/dea_results/svcd-mean_norm_counts.csv`: normalized counts and their means in each condition, from SVCD workflow.
* `data/blast2go/blast2go_go_table.txt`: result tables of Blast2GO Blast and GO annotation.
* `data/enrichment/gene_sets/go-basic_07OCT2019.obo`: file of GO terms such as a gene being annotated by a GO can also be annotated by the children of this GO.

## Output

* Raw results are with the folder `data/enrichment/results/go_background`.
* `data/enrichment/go_enrichment.xlsx`: synthesis excel sheet of functional enrichment results for the GO terms.

### Perform GSEA for GSEA software

* `data/enrichment/go_scores.rnk`: ranked list of KOs used to enrich GO gene set.
* `data/enrichment/gene_sets/go_background.gmx`: GO gene set from background of sequenced genes.

### Temporary files

* `data/temp/gsea_go.pkl`: results of GSEAPY for GO gene set.

In [1]:
import pandas as pd
import numpy as np
import pickle
import networkx as nx

import gseapy

In [2]:
import sys
from IPython.core.display import clear_output

def progress(cnt, k, N):
    """display a progress bar in the cell"""
    displayer = ['-', '\\', '|', '/']
    x = str( round(100*(k+1)/N, 2) ) + '%\t' + displayer[cnt]
    clear_output()
    sys.stdout.write(x)
    sys.stdout.flush()
    if cnt == 3: return 0
    else: return cnt + 1

In [3]:
dea_results = pd.read_csv( '../data/dea_results/synthesis_deg_results.csv', index_col=0 )
mean_counts = pd.read_csv( '../data/dea_results/svcd-mean_norm_counts.csv', index_col=0 )
b2g_results = pd.read_csv('../data/blast2go/blast2go_go_table.txt', sep='\t')

padj_thresh = 1e-4

In [4]:
# keep only genes with counts
b2g_mapped = b2g_results[ b2g_results['SeqName'].isin( mean_counts.index ) ]
# count the number of GO annotations per sequence
b2g_mapped = b2g_mapped[ b2g_mapped['Tags'] == '[BLASTED, MAPPED]' ]
b2g_mapped = b2g_mapped.set_index('SeqName')['GO IDs']

## Compute rank list for GO enrichment

In [5]:
go_scores = dea_results.copy()

# keep LFC with padj lower than the threshold
go_scores['LFC E'] = go_scores['LFC E'][ go_scores['padj E'] <= padj_thresh ]
go_scores['LFC S'] = go_scores['LFC S'][ go_scores['padj S'] <= padj_thresh ]
go_scores = go_scores[['LFC E', 'LFC S']].dropna(how='all')

# cumulated absolute LFC
go_scores = go_scores[['LFC E', 'LFC S']].abs().sum(axis=1)

print( 'number of genes:', len( go_scores ) )
print( 'score min - max:', go_scores.min(), '-', go_scores.max() )

# create the ranked list
to_write = '\n'.join([ id + '\t' + str( go_scores[id] )
                       for id in go_scores.index ])

number of genes: 5695
score min - max: 0.483034297229254 - 60.4378998490307


In [6]:
with open( '../data/enrichment/go_scores.rnk', 'w') as f: f.write( to_write )

## Build GO background list for enrichment
Following to Powell, J. A. C. (2014). GO2MSIG, an automated GO based multi-species gene set generator for gene set enrichment analysis. BMC Bioinformatics, 15(1), 146. doi:10.1186/1471-2105-15-146

### Get Blast2GO gene ontology mapping

In [7]:
goa = pd.DataFrame()

mapped_gos = b2g_mapped.str.split('; ').to_dict().items()
cnt, k, N = 0, 0, len(mapped_gos)
for gene, gos in mapped_gos:
    rows = pd.DataFrame([(g[0], g[2:]) for g in gos])
    rows['entry'] = gene
    goa = goa.append(rows)
    # progress
    cnt = progress(cnt, k, N)
    k += 1
    
goa = goa.replace( {'F': 'molecular_function', 'P': 'biological_process', 'C': 'cellular_component'}
                 ).rename( {0: 'namespace', 1: 'go'}, axis=1
                 ).drop_duplicates(
                 ).reset_index(drop=True)

100.0%	|

### Build tree of GOs from GO OBO file

In [ ]:
# load the GO database
with open( '../data/enrichment/gene_sets/go-basic_07OCT2019.obo', 'r' ) as f:
    lines = [line.rstrip() for line in f]

# parse the data
id = ''
all_gos, cnt, k, N = {}, 0, 0, len(lines)
for i, line in enumerate(lines):
    if line == '[Term]':
        id = lines[i+1].split('id: ')[1]
        all_gos[id] = {}
    elif id and line: 
        field = line.split(':')[0]
        value = ':'.join(line.split(': ')[1:])
        if field not in all_gos[id]: all_gos[id][ field ] = [ value ]
        else: all_gos[id][ field ] += [ value ]
    # progress
    cnt = progress(cnt, k, N)
    k += 1

99.87%	/

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
# create the tree of GOs; orientation GO1 (child) 'is_a' GO2 (parent) <=> GO1 -> GO2
go_tree = nx.DiGraph()
for i, go in enumerate(all_gos):
    # don't consider obsolete GOs
    not_obsolete = 'is_obsolete' not in all_gos[go] or all_gos[go]['is_obsolete'][0] != 'true'
    # make a relation between two GOs based only on 'is_a' (only the namespaces don't have 'is_a')
    if not_obsolete and 'is_a' in all_gos[go]:
        for isa in all_gos[go]['is_a']: go_tree.add_edge( go, isa.split(' ! ')[0] )
            
# flatten the tree to have: GO id => all its children + itself
go_flat = { go: nx.ancestors(go_tree, go) for go in go_tree }
go_flat = { go: tuple( children | {go} ) for go, children in go_flat.items() }

### Build background gene set

In [10]:
# get the entries associated to a given GO
association = { go: set( goa['entry'][goa['go'] == go].unique() )
                for go in goa['go'].unique() }

# complete with the entries from the children
association = { go: list(association[go] | { childs_entry
                                             for child in go_flat[go]
                                             if child in association
                                             for childs_entry in association[child] })
                for go in association if go in go_flat }

# search GOs having the same entries to remove duplicates and to avoid biases in enrichment
identical = { go1: [ go2 for go2 in association if association[go1] == association[go2] ]
              for go1 in association }
identical = { k: v for k, v in identical.items() if len(v) > 1 }

# create a common ontology and replace GOs with the same entries by a unique set
print( 'number of GOs without filtering:', len(association) )
for go in identical:
    if identical[go]:
        # common ontology (concatenated GO ids)
        common_go = ';'.join( identical[go] )
        association[ common_go ] = association[go]
        for i in identical[go]:
            del association[i] # remove the identical GOs
            identical[i] = '' # update identical
print( 'number of GOs after filtering:', len(association) )

number of GOs without filtering: 11694
number of GOs after filtering: 9269


In [11]:
def prepareGeneSet(dict_of_kos):
    """convert a dictionnary of elements into a GSEA gene set matrix"""
    keys = sorted( list(dict_of_kos.keys()) )
    # header
    to_write = '\t'.join( [ element for element in keys  ] ) + '\n'
    to_write += '\t'.join( [ 'na' for element in keys ] ) + '\n'
    # longuest list
    N = max([ len(dict_of_kos[element]) for element in keys ])
    # "transpose" the dictionary
    k = 0
    for i in range(N): # get the i-th element
        for j, element in enumerate( keys ): # of the j-th list
            if i < len( dict_of_kos[element] ): to_write += str(dict_of_kos[element][i])
            if j < len( dict_of_kos ) - 1: to_write += '\t'
        if i < N - 1: to_write += '\n'
    return to_write

with open('../data/enrichment/gene_sets/go_background.gmx', 'w') as f: f.write( prepareGeneSet(association) )

## Run GSEAPY

In [12]:
def run_gseapy( rnk, gmt, name ):
    # run gseapy
    results = gseapy.prerank( rnk, gmt, processes=10, seed=0, permutation_num=1000, min_size=5, max_size=1000,
                              no_plot=True, outdir='../data/enrichment/results/' + name
                   ).results
    # parse results
    cols = ['es', 'nes', 'pval', 'fdr', 'geneset_size', 'matched_size', 'genes']
    res = { p: [ results[p][k] for k in results[p] if k in cols ] for p in results }
    res = pd.DataFrame( res, index=cols ).T.sort_values( by='pval' ).sort_values( by='fdr' )
    res['genes'] = res['genes'].str.replace(';', '; ')
    res.columns = ['Enrichment score (ES)', 'Normalized enrichment score (NES)', 'p-value', 'FDR',
                   'Gene set size', 'Size after restriction to ranked list', 'Elements from ranked list']
    # keep positive results
    return res[ res['Enrichment score (ES)'] > 0 ]

In [13]:
res_go = run_gseapy( go_scores, association, 'go_background' )
go_naming = lambda x: all_gos[x]['name'][0] + ' ' + x
res_go = res_go.rename( { id:
                         go_naming(id) if len(id.split(';')) == 1 else 
                         '; '.join([ go_naming(i) for i in id.split(';') ])
                         for id in res_go.index } )

res_go.to_pickle('../data/temp/gsea_go.pkl')

In [14]:
res_go.to_pickle('../data/temp/gsea_go.pkl')
res_go.to_excel('../data/enrichment/go_enrichment.xlsx')